In [54]:
import pyspark
from pyspark.sql import Row
from pyspark import SQLContext

In [55]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [56]:
def inputValues(values):
    replace_new_line = values.replace('\n', ',')     # replace new line with comma
    input_value_to_list = replace_new_line.split(',')       # input values to list
    result = [x for x in input_value_to_list if x]      # remove empty space in list
    #result = [eval(i) for i in input_to_list]        # convert list from string to integer
    return result

In [57]:
sc = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

In [58]:
cars = sqlContext.read.format('com.github.saurfang.sas.spark').load(r'dataset/cars.sas7bdat')

In [59]:
cars.show(5)

+------+------------+-------+-----+------+---------+----------+---------+-------+------+---+---+------+
| Brand|       Model|Minivan|Wagon|Pickup|Automatic|EngineSize|Cylinders|CityMPG|HwyMPG|SUV|AWD|Hybrid|
+------+------------+-------+-----+------+---------+----------+---------+-------+------+---+---+------+
|TOYOTA|       Prius|    0.0|  0.0|   0.0|      1.0|       1.5|      4.0|   60.0|  51.0|0.0|0.0|   1.0|
| HONDA|Civic Hybrid|    0.0|  0.0|   0.0|      1.0|       1.3|      4.0|   48.0|  47.0|0.0|0.0|   1.0|
| HONDA|Civic Hybrid|    0.0|  0.0|   0.0|      1.0|       1.3|      4.0|   47.0|  48.0|0.0|0.0|   1.0|
| HONDA|Civic Hybrid|    0.0|  0.0|   0.0|      0.0|       1.3|      4.0|   46.0|  51.0|0.0|0.0|   1.0|
| HONDA|Civic Hybrid|    0.0|  0.0|   0.0|      0.0|       1.3|      4.0|   45.0|  51.0|0.0|0.0|   1.0|
+------+------------+-------+-----+------+---------+----------+---------+-------+------+---+---+------+
only showing top 5 rows



In [60]:
values = '''
TOYOTA
HONDA
VOLKSWAGEN
'''

searchdata = inputValues(values)

In [61]:
filtering = cars.filter(cars.Brand.isin(searchdata))
filtering = filtering.select(filtering['Brand'], filtering['Model'], filtering['EngineSize'])

In [62]:
filtering_sql = filtering.createOrReplaceTempView('filtering')

In [67]:
filtering_sql_1 = spark.sql('select * from filtering where EngineSize > 1.3') #Untuk export ke pandas
#spark.sql('select * from filtering where EngineSize > 1.3').show(50) #Hanya menampilkan table, tidak bisa export ke pandasabs

In [69]:
cars_df = filtering_sql_1.toPandas()

In [70]:
cars_df.head(5)

,Brand,Model,EngineSize
0,TOYOTA,Prius,1.5
1,VOLKSWAGEN,Golf,1.9
2,VOLKSWAGEN,Jetta,1.9
3,VOLKSWAGEN,New Beetle,1.9
4,HONDA,Civic,1.7


In [71]:
export = cars_df.to_excel('cars_filter.xlsx', index=False)